### This notebook
builds a graph-based embedding for a small number of random handwritten digits with multidimensional scaling.

Optional: run this before import to recompile c++ ops
```
!rm -r ../lib/cpp/temp/
!rm -r ../lib/cpp/build/
!rm -r ../lib/cpp/_bindings.so
!rm -r ../lib/cpp/bindings.py
!rm -r ../lib/cpp/lib_wrap.c
```


##### Get the dataset:

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "..")

from itertools import product, islice, chain
import math
import os.path
import time

import h5py
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD
from sklearn.manifold import MDS
import torch, torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as tqdm

torch.manual_seed(42)

import lib

In [ ]:
batch_size = 1024 # 64

In [ ]:
def chunks(iterable, size=10):
    iterator = iter(iterable)
    for first in iterator:
        yield chain([first], islice(iterator, size - 1))

In [ ]:
# Use standard FashionMNIST dataset
num_samples = 10000

_dim = 0  # assume square images, if not, use _dim1, _dim2
X, y = [], []

train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                             
    ])
)
loader = torch.utils.data.DataLoader(train_set, batch_size = num_samples)
for batch in loader:
    X, y = batch[0], batch[1]
    _dim = X.shape[-1]
    X = X.reshape(num_samples, _dim**2)
    X = X / np.square(X).sum(-1, keepdims=True) ** 0.5
    X = X.numpy()
# oops took the last batch by mistake, to fix

In [ ]:
if os.path.exists('./temp.hdf'):
    print("HDF5 file for pairwise distance for original dataset already exists, skipping this step!')
else:
    print("Generating pairwise distance for original dataset, writing to HDF5 file...")
    _generator = product(X, repeat=2)
    num_chunks = 100
    chunk_size = math.ceil(num_samples**2 / num_chunks)

    h5f = h5py.File('./temp.hdf', 'a')
    for _chunk in tqdm(chunks(_generator, chunk_size), total=num_chunks):
        distances = np.square([np.subtract(x[0], x[1]) for x in _chunk]).sum(-1)
        if "dataset" not in h5f:
            h5f_dataset = h5f.create_dataset('dataset', data=distances, compression="gzip", chunks=True, maxshape=(None, )) 
        else:
            h5f_dataset.resize((h5f_dataset.shape[0] + distances.shape[0]), axis = 0)
            h5f_dataset[-distances.shape[0]:] = distances
    h5f.close()

distances = []
with h5py.File('temp.hdf','r') as infile:
    distances = infile['dataset'][:]
distances = distances.reshape(num_samples, num_samples)

_elapsedTime = (time.time() - _start) / 60
print("Total time taken: ", _elapsedTime)
del _start, _elapsedTime

# 10000 samples **2 will take 20mins /w 100 chunks
    # hits around 9gb RAM usage, double 'num_chunks' if MemoryError
    # 350MB HDF5 file, 400MB RAM
    # 3 hr training time?
# 60000 samples **2 will take 36 times longer i.e. 12hr
    # probably 12.6gb HDF5 file, 14,4gb RAM

In [ ]:
# _storage = np.memmap('memmap.np', shape=(num_samples, _dim*_dim), dtype=float32, mode='wb+')

##### Build initial graph

We initialize prodige with a full graph initialized with distances between nodes

In [ ]:
from lib.task.compression import make_graph_from_vectors
emb = make_graph_from_vectors(
    X, knn_edges=100, max_length=10, n_jobs=-1, soft=True, directed=False, verbose=True
)

opt = torch.optim.SparseAdam(emb.parameters(), lr=0.01)

loss_history, reg_history = [], []

# uncomment to deliberately mess with weights for testing purposes
# emb.edge_weight_logits.reset_parameters()
# emb.edge_adjacency_logits.reset_parameters()

##### Training loop

Here we minimize the MDS loss function
$$L = 1/N \sum_{i, j} (d_{orig}(x_i, x_j) - d_G(v_i, v_j))^2$$

* $d_{orig}(x_i, x_j)$ is the original distance between two vectors in $X$
* $d_G(v_i, v_j)$ is the learned graph distance between corresponding vertices in graph $G$

In [ ]:
for batch_i in tqdm(range(len(loss_history), 650)): # 1000
    ii = torch.randint(0, len(X), [batch_size])
    jj = torch.randint(0, len(X), [batch_size])

    pred = emb(ii, jj)
    distances_ref = torch.as_tensor(distances[ii, jj], dtype=torch.float32)
    
    reconstruction_mse = F.mse_loss(pred['target_distances'], distances_ref)
    
    if len(loss_history) < 325: # 5000
        regularizer = emb.compute_l0_prior_penalty(batch_size=4096)
    else:
        regularizer = emb.compute_hierarchical_prior_penalty(nonzero_rate=0.05, batch_size=4096)

    lambd = min(1, len(loss_history) / 5000.) * 10.0
    loss = reconstruction_mse - pred['logp_target_paths'].mean() + lambd * regularizer
    opt.zero_grad()
    loss.backward()
    opt.step()
    loss_history.append(reconstruction_mse.item())
    reg_history.append(regularizer.item())
    
    if len(loss_history) % 10 == 0: # 50
        clear_output(True)
        plt.figure(figsize=[15, 4])
        plt.subplot(1, 3, 1);
        plt.title('reconstruction mse, mean = %0.5f' % np.mean(loss_history[-100:])); plt.grid()
        plt.plot(loss_history)
        
        plt.subplot(1, 3, 2);
        plt.title('regularizer, mean = %0.5f' % np.mean(reg_history[-100:])); plt.grid()
        plt.plot(reg_history)

        plt.subplot(1, 3, 3);
        probs = torch.sigmoid(emb.edge_adjacency_logits).data.numpy().flatten()
        nnz_rate = np.mean(probs > 0.5)
        plt.title('P(edge), nonzero rate = %.5f' % nnz_rate); plt.grid();
        plt.hist(probs)
        plt.show()

In [ ]:
print(emb)

In [ ]:
distances_ours = emb.compute_pairwise_distances()
print("PRODIGE: %.5f" % np.mean(np.square(distances - distances_ours)))

In [ ]:
if os.path.exists('./temp_pca.hdf'):
    print("HDF5 file for pairwise distance for PCA already exists, skipping this step!')
else:
    print("Generating pairwise distance for PCA, writing to HDF5 file...")

    pca = PCA(n_components=4).fit(X)
    ### pca = IncrementalPCA(n_components=4, batch_size=batch_size).fit(X)
    X_pca = pca.inverse_transform(pca.transform(X))

    _generator = product(X_pca, repeat=2)
    num_chunks = 100
    chunk_size = math.ceil(num_samples**2 / num_chunks)

    h5f = h5py.File('./temp_pca.hdf', 'a')
    for _chunk in tqdm(chunks(_generator, chunk_size), total=num_chunks):
        distances = np.square([np.subtract(x[0], x[1]) for x in _chunk]).sum(-1)
        if "dataset" not in h5f:
            h5f_dataset = h5f.create_dataset('dataset', data=distances, compression="gzip", chunks=True, maxshape=(None, )) 
        else:
            h5f_dataset.resize((h5f_dataset.shape[0] + distances.shape[0]), axis = 0)
            h5f_dataset[-distances.shape[0]:] = distances
    h5f.close()

distances_pca = []
with h5py.File('./temp_pca.hdf','r') as infile:
    distances_pca = infile['dataset'][:]
distances_pca = distances_pca.reshape(num_samples, num_samples)
print("PCA:\t %.5f" % np.mean(np.square(distances - distances_pca)))

In [ ]:
_start = time.time()

mds = MDS(n_components=4, random_state=42, n_jobs=2, dissimilarity='euclidean', verbose=1)  # 2 hrs
X_mds = mds.fit_transform(X)
distances_mds = np.square(X_mds[None, :, :] - X_mds[:, None, :]).sum(-1) ** 0.5
print("MDS:\t %.5f" % np.mean(np.square(distances - distances_mds)))

# svd = TruncatedSVD(n_components=4, algorithm='arpack')  # Very fast
# X_svd = svd.fit_transform(X)
# distances_svd = np.square(X_svd[None, :, :] - X_svd[:, None, :]).sum(-1) ** 0.5  # Probably causes memory issues
# print("SVD:\t %.5f" % np.mean(np.square(distances - distances_svd)))

_elapsedTime = (time.time() - _start) / 60
print("Total time taken: ", _elapsedTime)
del _start, _elapsedTime

In [ ]:
from bokeh import io
from bokeh.plotting import figure, output_file, save
output_file('mnist_graph.html')
emb.default_distance.data[...] = 100
p = lib.visualize_embeddings(emb, vertex_labels=y, edge_probability_threshold=0.5, deterministic=True,
                         vertex_alpha=0.8, cmap=lambda x: plt.get_cmap('rainbow')(x))
save(p, 'mnist_graph.html')

In [ ]:
from IPython.display import HTML
HTML(filename="graph.html")
# re-run several times to get different tsne optima

```

```

```

```

```

```

```

```

```

```
